###  Load Concept Mappings into a Database


####  Creat a data schema
        If a schema exists, then skip the next cell.
        Otherwise, uncomment the following spark.sql statement below:
                * create database <data schema name>

In [55]:
spark.sql("create database ontologies_concepts2")

AnalysisException: 'org.apache.hadoop.hive.metastore.api.AlreadyExistsException: Database ontologies_concepts2 already exists;'

In [56]:
spark.sql("use ontologies_concepts2").toPandas()

""


In [57]:
spark.sql("show tables").toPandas()

,database,tableName,isTemporary


In [58]:
#spark.sql("""select * from ontologies_concepts.mappings""").printSchema()

In [59]:
spark.sql("""select * from ontologies_concepts2.conceptmaps""").printSchema()

AnalysisException: "Table or view not found: `ontologies_concepts2`.`conceptmaps`; line 1 pos 14;\n'Project [*]\n+- 'UnresolvedRelation `ontologies_concepts2`.`conceptmaps`\n"

In [ ]:
### remove the comment, if the ontologies_concepts does not exists
#spark.sql("create database ontologies_concepts")

#### Set the environment variables
        * set the contextNumber to your context number
        * set the version to the version of the context that you want
        * set the root directory to your S3 bucket  
            (i.e. for consult, it is s3://consult-datalab-persistence-s3-data/discernontology/v1/)

In [ ]:
####  See Amazon S3consult-datalab-persistence-s3-data/discernontology/v1/versionedcontexts

In [60]:
contextNumber = '454C2298850B455CA513888EB1FD8BCF'
contextNumber = contextNumber.upper().replace("-", "")
print(contextNumber)

version=8
print(version)

# Location of the discernontology database
root = 's3://consult-datalab-persistence-s3-data/discernontology/v1/'
print(root)

454C2298850B455CA513888EB1FD8BCF
8
s3://consult-datalab-persistence-s3-data/discernontology/v1/


#### Verify that the context exists or not 

In [65]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
spark.sql("""select m.targetValue as concept, 
                    right(m.conceptMapUri,32) as context,
                    max(m.conceptmapversion) as version  
                    from ontologies_concepts2.mappings m
                 where conceptMapUri like '%{CN}' 
                   
                 group by m.targetValue, m.conceptMapUri 
                  LIMIT 150""".format(**{
        'CN':contextNumber
    })).toPandas()  


,concept,context,version
0,CELLULITIS_CLIN,454C2298850B455CA513888EB1FD8BCF,0008
1,TPA_WITHIN_24_HOURS_CLIN,454C2298850B455CA513888EB1FD8BCF,0008
2,PERCUTANEOUS_CORONARY_INTERVENTION_PROC,454C2298850B455CA513888EB1FD8BCF,0008
3,SEPSIS_CLIN,454C2298850B455CA513888EB1FD8BCF,0008
4,PERCUTANEOUS_CV_PROC_WITHOUT_CORONARY_ARTERY_STENT_PROC,454C2298850B455CA513888EB1FD8BCF,0008
5,LOWER_AND_HUMERUS_PROCEDURE_EXCEPT_HIP_FOOT_FEMUR_PROC,454C2298850B455CA513888EB1FD8BCF,0008
6,CERVICAL_SPINAL_FUSION_PROC,454C2298850B455CA513888EB1FD8BCF,0008
7,STENT_NOT_DRUG_ELUTING_PROC,454C2298850B455CA513888EB1FD8BCF,0008
8,MAJOR_JOINT_REPLACEMENT_LE_KNEE_PROC,454C2298850B455CA513888EB1FD8BCF,0008
9,STROKE_ISCHEMIC_PRECEREBRAL_OCCLUSION_OR_TRANSIENT_ISCHEMIA_CLIN,454C2298850B455CA513888EB1FD8BCF,0008


#### Load the contexts to your data schema
##### If nothing is return from above, then run the following

In [62]:
from foresight.discern import with_discern_context
from bunsen.stu3.codes import create_concept_maps

maps = create_concept_maps(spark)

maps = with_discern_context(spark,
                           maps,
                           root,
                           contextNumber, 
                           version)

maps.write_to_database('ontologies_concepts2')


In [73]:

pandasDF = spark.sql("""select m.targetValue as concept, 
                    right(m.conceptMapUri,32) as context,
                   -- max(m.conceptmapversion) as version, 
                    m.*
                    from ontologies_concepts2.mappings m
                 where conceptMapUri like '%{CN}' 
                 
               --  group by m.targetValue, m.conceptMapUri 
                 order by 1  """.format(**{
        'CN':contextNumber
    })).toPandas()  


In [74]:
pandasDF.to_excel('conceptMappings.xlsx', encoding='utf-8')

In [75]:
pandasDF.count()

concept              6050
context              6050
sourceValueSet       6050
targetValueSet       6050
sourceSystem         6050
sourceValue          6050
targetSystem         6050
targetValue          6050
equivalence          6050
conceptmapuri        6050
conceptmapversion    6050
dtype: int64

In [76]:
spark.sql("""select * from ontologies_concepts2.mappings
where
conceptMapUri like '%454C2298850B455CA513888EB1FD8BCF%'
""").limit(15).toPandas()

,sourceValueSet,targetValueSet,sourceSystem,sourceValue,targetSystem,targetValue,equivalence,conceptmapuri,conceptmapversion
0,urn:valueset:oid:2.16.840.1.113883.6.90,urn:cerner:foresight:valueset:discern-ontologies-placeholder,http://hl7.org/fhir/sid/icd-10-cm,I21.9,urn:cerner:foresight:system:discern-ontologies-placeholder,ACUTE_MYOCARDIAL_INFARCTION_CLIN,subsumes,urn:cerner:foresight:mapping:discern-ontologies:454C2298850B455CA513888EB1FD8BCF,0008
1,urn:valueset:oid:2.16.840.1.113883.6.90,urn:cerner:foresight:valueset:discern-ontologies-placeholder,http://hl7.org/fhir/sid/icd-10-cm,I21.4,urn:cerner:foresight:system:discern-ontologies-placeholder,ACUTE_MYOCARDIAL_INFARCTION_CLIN,subsumes,urn:cerner:foresight:mapping:discern-ontologies:454C2298850B455CA513888EB1FD8BCF,0008
2,urn:valueset:oid:2.16.840.1.113883.6.90,urn:cerner:foresight:valueset:discern-ontologies-placeholder,http://hl7.org/fhir/sid/icd-10-cm,I21.01,urn:cerner:foresight:system:discern-ontologies-placeholder,ACUTE_MYOCARDIAL_INFARCTION_CLIN,subsumes,urn:cerner:foresight:mapping:discern-ontologies:454C2298850B455CA513888EB1FD8BCF,0008
3,urn:valueset:oid:2.16.840.1.113883.6.90,urn:cerner:foresight:valueset:discern-ontologies-placeholder,http://hl7.org/fhir/sid/icd-10-cm,I22.8,urn:cerner:foresight:system:discern-ontologies-placeholder,ACUTE_MYOCARDIAL_INFARCTION_CLIN,subsumes,urn:cerner:foresight:mapping:discern-ontologies:454C2298850B455CA513888EB1FD8BCF,0008
4,urn:valueset:oid:2.16.840.1.113883.6.90,urn:cerner:foresight:valueset:discern-ontologies-placeholder,http://hl7.org/fhir/sid/icd-10-cm,I22.0,urn:cerner:foresight:system:discern-ontologies-placeholder,ACUTE_MYOCARDIAL_INFARCTION_CLIN,subsumes,urn:cerner:foresight:mapping:discern-ontologies:454C2298850B455CA513888EB1FD8BCF,0008
5,urn:valueset:oid:2.16.840.1.113883.6.90,urn:cerner:foresight:valueset:discern-ontologies-placeholder,http://hl7.org/fhir/sid/icd-10-cm,I21.02,urn:cerner:foresight:system:discern-ontologies-placeholder,ACUTE_MYOCARDIAL_INFARCTION_CLIN,subsumes,urn:cerner:foresight:mapping:discern-ontologies:454C2298850B455CA513888EB1FD8BCF,0008
6,urn:valueset:oid:2.16.840.1.113883.6.90,urn:cerner:foresight:valueset:discern-ontologies-placeholder,http://hl7.org/fhir/sid/icd-10-cm,I21.A9,urn:cerner:foresight:system:discern-ontologies-placeholder,ACUTE_MYOCARDIAL_INFARCTION_CLIN,subsumes,urn:cerner:foresight:mapping:discern-ontologies:454C2298850B455CA513888EB1FD8BCF,0008
7,urn:valueset:oid:2.16.840.1.113883.6.90,urn:cerner:foresight:valueset:discern-ontologies-placeholder,http://hl7.org/fhir/sid/icd-10-cm,I21.3,urn:cerner:foresight:system:discern-ontologies-placeholder,ACUTE_MYOCARDIAL_INFARCTION_CLIN,subsumes,urn:cerner:foresight:mapping:discern-ontologies:454C2298850B455CA513888EB1FD8BCF,0008
8,urn:valueset:oid:2.16.840.1.113883.6.90,urn:cerner:foresight:valueset:discern-ontologies-placeholder,http://hl7.org/fhir/sid/icd-10-cm,I21.21,urn:cerner:foresight:system:discern-ontologies-placeholder,ACUTE_MYOCARDIAL_INFARCTION_CLIN,subsumes,urn:cerner:foresight:mapping:discern-ontologies:454C2298850B455CA513888EB1FD8BCF,0008
9,urn:valueset:oid:2.16.840.1.113883.6.90,urn:cerner:foresight:valueset:discern-ontologies-placeholder,http://hl7.org/fhir/sid/icd-10-cm,I22.2,urn:cerner:foresight:system:discern-ontologies-placeholder,ACUTE_MYOCARDIAL_INFARCTION_CLIN,subsumes,urn:cerner:foresight:mapping:discern-ontologies:454C2298850B455CA513888EB1FD8BCF,0008


In [ ]:
spark.sql("""select * from ontologies_concepts2.conceptmaps
where
url like '%454C2298850B455CA513888EB1FD8BCF%'
""").limit(15).toPandas()

In [ ]:
spark.sql("""select * from ontologies_concepts2.conceptmaps
""").count()

In [66]:
conceptmapsDF = spark.sql("""select * from ontologies_concepts2.conceptmaps
where
url like '%454C2298850B455CA513888EB1FD8BCF%'
""")

In [67]:
conceptmapsDF.count()

1

In [68]:
conceptmapsDF.toPandas()

,id,meta,implicitRules,language,text,url,identifier,version,name,title,...,useContext,jurisdiction,purpose,copyright,sourceUri,sourceReference,targetUri,targetReference,group,timestamp
0,None,"(None, None, None, [], [], [])",None,None,"(None, None, None)",urn:cerner:foresight:mapping:discern-ontologies:454C2298850B455CA513888EB1FD8BCF,"(None, None, (None, [], None), None, None, (None, None, None), (None, None))",0008,None,None,...,[],[],None,None,None,None,None,None,[],2019-07-08 16:26:08.268


In [ ]:
spark.sql("""DROP TABLE IF EXISTS ontologies_concepts2.conceptmaps""").show()

In [ ]:
spark.sql("""DROP TABLE IF EXISTS ontologies_concepts2.mappings""").show()